In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import pandas as pd
import seaborn as sns

# Import Data

In [ ]:
%%shell
cd /content
gdown -q --id "1L_9NzgppycZwQgK6UvHjpQJS-bzNMfot"

In [ ]:
df = pd.read_csv("real-estate-content-8.csv")
print(df.head())

                                                 url  ...    website
0  https://nhadat247.com.vn/cho-thue-van-phong/ch...  ...  nhadat247
1  https://nhadat247.com.vn/cho-thue-nha-mat-pho/...  ...  nhadat247
2  https://nhadat247.com.vn/cho-thue-van-phong/ca...  ...  nhadat247
3  https://nhadat247.com.vn/cho-thue-can-ho-chung...  ...  nhadat247
4  https://nhadat247.com.vn/cho-thue-can-ho-chung...  ...  nhadat247

[5 rows x 47 columns]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19137 entries, 0 to 19136
Data columns (total 47 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   19137 non-null  object 
 1   post_title            19137 non-null  object 
 2   price_unit            19137 non-null  object 
 3   price                 19137 non-null  object 
 4   total_site_area       19137 non-null  object 
 5   property_name         19137 non-null  object 
 6   address_id            19137 non-null  object 
 7   number_of_bedrooms    11758 non-null  object 
 8   number_of_bathrooms   451 non-null    object 
 9   project_id            19137 non-null  object 
 10  project_size          19137 non-null  object 
 11  post_author           19137 non-null  object 
 12  property_code         19137 non-null  int64  
 13  full_description      19137 non-null  object 
 14  phone_number          19137 non-null  object 
 15  email              

**SELECT FEATURES**

In [ ]:
df2 = df[[ 'total_site_area', 'property_name', 'number_of_bedrooms','city', 'district' ,'price_unit' ]]
print(df2.head())
print(df2.tail())
print(df2.shape)

  total_site_area       property_name  ...       district       price_unit
0           60 m²  cho thuê văn phòng  ...      Hoàng Mai    6 Triệu/tháng
1          110 m²           Nhà riêng  ...        Đống Đa   60 Triệu/tháng
2          700 m²  cho thuê văn phòng  ...         Quận 2  158 Triệu/tháng
3           80 m²              Căn hộ  ...   Hai Bà Trưng   10 Triệu/tháng
4           89 m²              Căn hộ  ...    Bắc Từ Liêm   13 Triệu/tháng

[5 rows x 6 columns]
      total_site_area property_name  ... district      price_unit
19132           88 m²        Căn hộ  ...   Quận 2  19 triệu/tháng
19133           40 m²     Nhà riêng  ...   Tây Hồ  15 triệu/tháng
19134          100 m²        Căn hộ  ...   Quận 4  19 triệu/tháng
19135           60 m²        Căn hộ  ...   Quận 9   6 triệu/tháng
19136           74 m²        Căn hộ  ...   Quận 4  15 triệu/tháng

[5 rows x 6 columns]
(19137, 6)


**CLEAN NULL DATA**

In [ ]:
print(df2.isnull().sum())
df2 = df2[~(df2.price_unit == ' ')]
df2 = df2[~(df2.total_site_area == ' ')]
df2 = df2.dropna()


total_site_area          0
property_name            0
number_of_bedrooms    7379
city                     0
district                 0
price_unit               0
dtype: int64


In [ ]:
print(df2.isnull().sum())
print(len(df2))

total_site_area       0
property_name         0
number_of_bedrooms    0
city                  0
district              0
price_unit            0
dtype: int64
11014


**Preprocessing price_unit**

In [ ]:
text_unique = set()
for obj in df2['price_unit']:
  text_unique.add(obj.split(' ')[1])
print(text_unique)
print(len(df2['price_unit']))
# text_unique = df2['price_unit']
# print(text_unique.split(' ')[1])

{'nghìn/m2/tháng', 'thuận', 'Triệu/tháng', 'triệu/tháng', 'triệu/m²', 'triệu/m2/tháng', 'Triệu/m²'}
11014


In [ ]:
indexes_to_drop = []
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
for index in range (left_size):
  if(df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'nghìn/m2/tháng'):
    area = df2['total_site_area'][df2_left_indexes[index]]
    final_data =  df2['price_unit'][df2_left_indexes[index]].split(' ')[0]
    leng = len(final_data)
    final_data = float(final_data) * pow(10,-leng)
    area = float(area.split(' ')[0])
    final_data = final_data*area
    df2['price_unit'][df2_left_indexes[index]] = final_data
  elif (df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'triệu/m2/tháng'):
    area = df2['total_site_area'][df2_left_indexes[index]]
    final_data =  df2['price_unit'][df2_left_indexes[index]].split(' ')[0]
    final_data = float(final_data)
    area = float(area.split(' ')[0])
    final_data = final_data*area
    df2['price_unit'][df2_left_indexes[index]] = final_data
  elif (df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'Triệu/m²' or  df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'nghìn/m²' or df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'triệu/m²' or df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'thuận' or df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == ' ' ):
    indexes_to_drop.append(index)
  elif (df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'nghìn/tháng'):
    final_data =  df2['price_unit'][df2_left_indexes[index]].split(' ')[0]
    leng = len(final_data)
    final_data = float(final_data) * pow(10,-leng)
    df2['price_unit'][df2_left_indexes[index]] = final_data
  elif (df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'Triệu/tháng' or df2['price_unit'][df2_left_indexes[index]].split(' ')[1] == 'triệu/tháng' ):
    final_data =  df2['price_unit'][df2_left_indexes[index]].split(' ')[0]
    df2['price_unit'][df2_left_indexes[index]] = float(final_data)

indexes_to_keep = set(range(df2.shape[0])) - set(indexes_to_drop)
df2 = df2.take(list(indexes_to_keep))



**Preprocessing house size**

In [ ]:
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
for index in range (left_size):
  df2['total_site_area'][df2_left_indexes[index]] = float(df2['total_site_area'][df2_left_indexes[index]].split(' ')[0])
print(df2['total_site_area'].unique())


[55.0 88.0 72.0 78.0 74.0 54.0 94.0 63.0 85.0 50.0 76.0 100.0 60.0 135.0
 90.0 108.3 80.0 123.0 70.0 45.0 91.0 98.0 59.0 115.0 86.0 35.0 95.0 50.33
 30.0 145.0 160.0 71.0 120.0 112.0 69.0 65.0 102.0 200.0 187.0 75.0 49.0
 800.0 110.0 56.0 66.38 51.0 34.0 40.0 87.0 64.0 1000.0 16.0 67.0 38.0
 73.0 105.0 320.0 90.78 43.0 81.0 96.0 109.0 68.0 150.0 600.0 384.0 47.0
 107.0 175.0 125.0 82.0 140.0 53.0 92.0 141.0 29.0 28.0 25.0 84.0 162.0
 153.0 77.0 89.0 130.0 124.0 83.0 146.0 256.0 78.42 79.0 42.0 57.0 37.0
 48.0 66.0 127.0 52.0 185.0 108.0 97.0 128.0 111.0 72.25 134.0 126.0 104.0
 58.8 99.0 300.0 157.5 448.0 67.5 143.0 61.0 116.0 132.0 62.27 101.0 36.0
 32.0 240.0 33.0 180.0 155.0 129.0 118.0 216.0 27.5 58.0 106.0 100.2 103.0
 250.0 380.0 62.0 113.0 181.0 95.38 60.48 73.9 500.0 122.0 212.5 310.0
 103.5 38.4 138.0 273.0 72.95 46.0 340.0 144.0 94.6 220.0 161.0 148.7
 450.0 67.58 27.0 72.45 360.0 69.7 1500.0 2000.0 152.0 67.8 170.0 69.9
 31.0 1125.0 98.9 133.0 117.0 18.0 114.0 171.0 550.0 40

In [ ]:
df2 = df2[~(df2.total_site_area == 'm²')]
print(df2['total_site_area'].unique())


[55.0 88.0 72.0 78.0 74.0 54.0 94.0 63.0 85.0 50.0 76.0 100.0 60.0 135.0
 90.0 108.3 80.0 123.0 70.0 45.0 91.0 98.0 59.0 115.0 86.0 35.0 95.0 50.33
 30.0 145.0 160.0 71.0 120.0 112.0 69.0 65.0 102.0 200.0 187.0 75.0 49.0
 800.0 110.0 56.0 66.38 51.0 34.0 40.0 87.0 64.0 1000.0 16.0 67.0 38.0
 73.0 105.0 320.0 90.78 43.0 81.0 96.0 109.0 68.0 150.0 600.0 384.0 47.0
 107.0 175.0 125.0 82.0 140.0 53.0 92.0 141.0 29.0 28.0 25.0 84.0 162.0
 153.0 77.0 89.0 130.0 124.0 83.0 146.0 256.0 78.42 79.0 42.0 57.0 37.0
 48.0 66.0 127.0 52.0 185.0 108.0 97.0 128.0 111.0 72.25 134.0 126.0 104.0
 58.8 99.0 300.0 157.5 448.0 67.5 143.0 61.0 116.0 132.0 62.27 101.0 36.0
 32.0 240.0 33.0 180.0 155.0 129.0 118.0 216.0 27.5 58.0 106.0 100.2 103.0
 250.0 380.0 62.0 113.0 181.0 95.38 60.48 73.9 500.0 122.0 212.5 310.0
 103.5 38.4 138.0 273.0 72.95 46.0 340.0 144.0 94.6 220.0 161.0 148.7
 450.0 67.58 27.0 72.45 360.0 69.7 1500.0 2000.0 152.0 67.8 170.0 69.9
 31.0 1125.0 98.9 133.0 117.0 18.0 114.0 171.0 550.0 40

## property_name preprocessing - labeling

In [ ]:
df2['property_name'].unique()

array([3, 2, 1, 4, 0], dtype=object)

In [ ]:
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
empty_list = []
for index in range(left_size):
  if(df2['property_name'][df2_left_indexes[index]] == ' '):
    empty_list.append(index)
print(empty_list)
print(len(empty_list))
# print(set(range(df2.shape[0])))


[]
0


In [ ]:
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
for index in range(left_size):
  if(df2['property_name'][df2_left_indexes[index]] == 'cho thuê văn phòng' or df2['property_name'][df2_left_indexes[index]] == 'Cho thuê văn phòng' ):
    df2['property_name'][df2_left_indexes[index]] = 0
  elif(df2['property_name'][df2_left_indexes[index]] == 'Cho thuê cửa hàng, ki ốt' ):
    df2['property_name'][df2_left_indexes[index]] = 1
  elif(df2['property_name'][df2_left_indexes[index]] == 'Nhà riêng' ):
    df2['property_name'][df2_left_indexes[index]] = 2
  elif(df2['property_name'][df2_left_indexes[index]] == 'Căn hộ' ):
    df2['property_name'][df2_left_indexes[index]] = 3
  elif(df2['property_name'][df2_left_indexes[index]] == 'Đất nền' ):
    df2['property_name'][df2_left_indexes[index]] = 4

print(df2['property_name'].unique())

[3 2 1 4 0]


## Number_Of_Bedroom

In [ ]:
df2 = df2[~(df2.number_of_bedrooms == ' ')]
# df2_left_indexes = list(df2.index.values)
# left_size = len(df2_left_indexes)
# print(left_size)
# empty_list = []
# for index in range(left_size):
#   if(df2['number_of_bedrooms'][df2_left_indexes[index]] == ' '):
#     empty_list.append(index)
# print(empty_list)
# print(len(empty_list))

In [ ]:
# indexes_to_keep = set(range(df2.shape[0])) - set(empty_list)
# indexes_to_keep = list(indexes_to_keep)
# df2 = df2.take(indexes_to_keep)

In [ ]:
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
for index in range (left_size):
  df2['number_of_bedrooms'][df2_left_indexes[index]] = float(df2['number_of_bedrooms'][df2_left_indexes[index]])
print(len(df2))
print(df2['number_of_bedrooms'].unique())

10454
[1.0 2.0 3.0 5.0 4.0 7.0 6.0 8.0 18.0 13.0 29.0 14.0 15.0 9.0 11.0 27.0
 52.0 22.0 10.0 17.0 21.0 86.0 12.0 28.0 53.0 20.0 41.0 50.0 30.0 34.0
 16.0 23.0 25.0 73.0 40.0]


**Preprocessing location(district + city)**

In [ ]:

df2  = df2[~(df2.city == ' ')]
df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
print(df2['city'].unique())
for index in range (left_size):
  if(df2['city'][df2_left_indexes[index]] == ' Hồ Chí Minh' or df2['city'][df2_left_indexes[index]]  == ' Hồ Chí Minh.' or df2['city'][df2_left_indexes[index]]  == ' Tp. HCM' or df2['city'][df2_left_indexes[index]]  == ' hồ chí minh'):
    df2['city'][df2_left_indexes[index]]  = 'TP.Hồ Chí Minh'
  elif(df2['city'][df2_left_indexes[index]] == ' Hà Nội.' or df2['city'][df2_left_indexes[index]] == ' Hà Nội' or df2['city'][df2_left_indexes[index]] == ' Hà Nội' or df2['city'][df2_left_indexes[index]] == ' hồ chí minh'):
    df2['city'][df2_left_indexes[index]]  = 'Hà Nội'
  elif(df2['city'][df2_left_indexes[index]] == ' Đà Nẵng' or df2['city'][df2_left_indexes[index]] == ' TP đà Nẵng'):
    df2['city'][df2_left_indexes[index]] = 'Đà Nẵng'
  elif(df2['city'][df2_left_indexes[index]] == ' Bình Dương' or df2['city'][df2_left_indexes[index]] == ' Bình Dương.' ):
    df2['city'][df2_left_indexes[index]] = 'Bình Dương'
  elif(df2['city'][df2_left_indexes[index]] == ' Bà Rịa Vũng Tàu' or df2['city'][df2_left_indexes[index]] == ' Bà Rịa Vũng Tàu.'):
    df2['city'][df2_left_indexes[index]] = 'Bà Rịa Vũng Tàu'
  elif(df2['city'][df2_left_indexes[index]] == ' Cần Thơ.'):
    df2['city'][df2_left_indexes[index]] = ' Cần Thơ'


['TP.Hồ Chí Minh' 'Hà Nội' 'Đà Nẵng' 'Bình Dương' 'Bà Rịa Vũng Tàu'
 ' Hải Phòng' ' Bắc Ninh' ' Khánh Hòa' ' Đồng Nai' ' Cần Thơ' ' Hưng Yên'
 ' Kiên Giang' ' Long An' ' Tiền Giang' ' Quảng Ninh' ' Bình Định'
 ' Lâm Đồng' ' Bình Thuận  ' ' Thừa Thiên Huế' ' Quảng Nam']


In [ ]:
print(df2['city'].unique())

['TP.Hồ Chí Minh' 'Hà Nội' 'Đà Nẵng' 'Bình Dương' 'Bà Rịa Vũng Tàu'
 ' Hải Phòng' ' Bắc Ninh' ' Khánh Hòa' ' Đồng Nai' ' Cần Thơ' ' Hưng Yên'
 ' Kiên Giang' ' Long An' ' Tiền Giang' ' Quảng Ninh' ' Bình Định'
 ' Lâm Đồng' ' Bình Thuận  ' ' Thừa Thiên Huế' ' Quảng Nam']


In [ ]:

print(df2['district'].unique())

[' Quận 1' ' Quận 9' ' Nam Từ Liêm' ' Thủ Đức' ' Quận 4' ' Quận 7'
 ' Thanh Khê' ' Bình Tân' ' Hà Đông' ' Thủ Dầu Một' ' Thanh Xuân'
 ' Quận 2' ' Bình Thạnh' ' Vũng Tàu' ' Cầu Giấy' ' Tây Hồ' ' Quận 10'
 ' Tân Bình' ' Quận 3' ' Hoài Đức' ' Gia Lâm' ' Phú Nhuận' ' Quận 5'
 ' Hai Bà Trưng' ' Gò Vấp' ' Tân Phú' ' Hồng Bàng' ' Bắc Từ Liêm'
 ' Quận 6' 'Tân Bình' ' Đống Đa' ' Bình Chánh' ' Hoàn Kiếm' ' Dĩ An'
 ' Nhà Bè' ' Long Biên' ' Quận 8' ' Hoàng Mai' ' Quận 12' ' Bắc Ninh'
 ' Nha Trang' ' Biên Hòa' ' Quận 11' ' Ba Đình' 'Ngô Quyền'
 ' Ngũ Hành Sơn' ' Sơn Trà' ' Ngô Quyền' ' Lê Chân' ' Cái Răng'
 ' Thuận An' ' Thanh Trì' ' Ninh Kiều' ' Văn Giang' ' Phú Quốc' ' Bến Lức'
 ' Nhơn Trạch' ' Hải Châu' 'Quận 7' ' Mỹ Tho' 'Quận 8' ' Hạ Long'
 ' Liên Chiểu' ' Hải An' ' Hóc Môn' ' Quy Nhơn' ' Từ Sơn' 'Quận 3'
 ' Bà Rịa' ' An Dương' 'Quận 10' ' Đà Lạt' ' Đức Hòa' ' Đông Anh'
 ' Bình Thủy' 'Quận 2' 'Long Biên' ' Đan Phượng' ' Bến Cát' 'Phước Ninh'
 ' Dương Kinh' 'Sơn Trà' 'Đường Chu Văn An quận Bình

In [ ]:

df2_left_indexes = list(df2.index.values)
left_size = len(df2_left_indexes)
for index in range (left_size):
  if(df2['district'][df2_left_indexes[index]] == ' Q. 1' or df2['district'][df2_left_indexes[index]] == 'Quận 1' ):
    df2['district'][df2_left_indexes[index]]  = ' Quận 1'
  elif(df2['district'][df2_left_indexes[index]] == ' Quận 7' ):
    df2['district'][df2_left_indexes[index]]  = 'Quận 7'
  elif(df2['district'][df2_left_indexes[index]] == ' thủ đức' ):
    df2['district'][df2_left_indexes[index]]  = ' Thủ Đức'
  elif(df2['district'][df2_left_indexes[index]] == ' Quận 2' ):
    df2['district'][df2_left_indexes[index]]  = 'Quận 2'
  elif(df2['district'][df2_left_indexes[index]] == 'Đường Chu Văn An quận Bình Thạnh' ):
    df2['district'][df2_left_indexes[index]]  = ' Bình Thạnh'
  elif(df2['district'][df2_left_indexes[index]] == 'Đường Chu Văn An quận Bình Thạnh' ):
    df2['district'][df2_left_indexes[index]]  = ' Bình Thạnh'
  elif(df2['district'][df2_left_indexes[index]] == ' Quận 10' ):
    df2['district'][df2_left_indexes[index]]  = 'Quận 10'
  elif(df2['district'][df2_left_indexes[index]] == ' Tân Bình' ):
    df2['district'][df2_left_indexes[index]]  = 'Tân Bình'
  elif(df2['district'][df2_left_indexes[index]] == ' Quận 3' ):
    df2['district'][df2_left_indexes[index]]  = 'Quận 3'
  elif(df2['district'][df2_left_indexes[index]] == ' Tân Phú' ):
    df2['district'][df2_left_indexes[index]]  = 'Tân Phú'
  elif(df2['district'][df2_left_indexes[index]] == ' Long Biên' ):
    df2['district'][df2_left_indexes[index]]  = 'Long Biên'
  elif(df2['district'][df2_left_indexes[index]] == ' Quận 8' ):
    df2['district'][df2_left_indexes[index]]  = 'Quận 8'
  elif(df2['district'][df2_left_indexes[index]] == ' Ngô Quyền' ):
    df2['district'][df2_left_indexes[index]]  = 'Ngô Quyền'
  elif(df2['district'][df2_left_indexes[index]] == ' Sơn Trà' ):
    df2['district'][df2_left_indexes[index]]  = 'Sơn Trà'
  elif(df2['district'][df2_left_indexes[index]] == ' Lê Chân' ):
    df2['district'][df2_left_indexes[index]]  = 'Lê Chân'


In [ ]:

print(df2['district'].unique())

[' Quận 1' ' Quận 9' ' Nam Từ Liêm' ' Thủ Đức' ' Quận 4' 'Quận 7'
 ' Thanh Khê' ' Bình Tân' ' Hà Đông' ' Thủ Dầu Một' ' Thanh Xuân' 'Quận 2'
 ' Bình Thạnh' ' Vũng Tàu' ' Cầu Giấy' ' Tây Hồ' 'Quận 10' 'Tân Bình'
 'Quận 3' ' Hoài Đức' ' Gia Lâm' ' Phú Nhuận' ' Quận 5' ' Hai Bà Trưng'
 ' Gò Vấp' 'Tân Phú' ' Hồng Bàng' ' Bắc Từ Liêm' ' Quận 6' ' Đống Đa'
 ' Bình Chánh' ' Hoàn Kiếm' ' Dĩ An' ' Nhà Bè' 'Long Biên' 'Quận 8'
 ' Hoàng Mai' ' Quận 12' ' Bắc Ninh' ' Nha Trang' ' Biên Hòa' ' Quận 11'
 ' Ba Đình' 'Ngô Quyền' ' Ngũ Hành Sơn' 'Sơn Trà' 'Lê Chân' ' Cái Răng'
 ' Thuận An' ' Thanh Trì' ' Ninh Kiều' ' Văn Giang' ' Phú Quốc' ' Bến Lức'
 ' Nhơn Trạch' ' Hải Châu' ' Mỹ Tho' ' Hạ Long' ' Liên Chiểu' ' Hải An'
 ' Hóc Môn' ' Quy Nhơn' ' Từ Sơn' ' Bà Rịa' ' An Dương' ' Đà Lạt'
 ' Đức Hòa' ' Đông Anh' ' Bình Thủy' ' Đan Phượng' ' Bến Cát' 'Phước Ninh'
 ' Dương Kinh' ' Phan Thiết' ' Rạch Giá' ' Tân Uyên' ' Huế' ' La Gi'
 ' Hội An' ' Chương Mỹ' ' Thủy Nguyên']


In [ ]:
df2[['city','district']]
df2["location"] = df2["district"] + " " + df2["city"]
print(df2["location"])

361        Quận 1 TP.Hồ Chí Minh
374        Quận 9 TP.Hồ Chí Minh
376           Nam Từ Liêm Hà Nội
382        Quận 9 TP.Hồ Chí Minh
385       Thủ Đức TP.Hồ Chí Minh
                  ...           
19132      Quận 2 TP.Hồ Chí Minh
19133              Tây Hồ Hà Nội
19134      Quận 4 TP.Hồ Chí Minh
19135      Quận 9 TP.Hồ Chí Minh
19136      Quận 4 TP.Hồ Chí Minh
Name: location, Length: 10454, dtype: object


**Remove those locations that have small replication**

In [ ]:
len(df2['location'].unique())

81

In [ ]:
df2.location = df2.location.apply(lambda x: x.strip())
location_stats = df2.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
Quận 2 TP.Hồ Chí Minh        980
Bình Thạnh TP.Hồ Chí Minh    808
Quận 7 TP.Hồ Chí Minh        734
Nam Từ Liêm Hà Nội           716
Cầu Giấy Hà Nội              547
                            ... 
Bến Lức  Long An               1
Chương Mỹ Hà Nội               1
Phú Quốc  Kiên Giang           1
Mỹ Tho  Tiền Giang             1
Huế  Thừa Thiên Huế            1
Name: location, Length: 81, dtype: int64

In [ ]:
len(location_stats[location_stats <= 10])

32

In [ ]:
#Locations that have small number of datapoints
location_stats_less_than_10 = location_stats[location_stats <= 10]
location_stats_less_than_10

location
Ninh Kiều  Cần Thơ        10
Bắc Ninh  Bắc Ninh         9
Hóc Môn TP.Hồ Chí Minh     9
Ngũ Hành Sơn Đà Nẵng       8
Cái Răng  Cần Thơ          7
Hải An  Hải Phòng          7
Liên Chiểu Đà Nẵng         5
Đà Lạt  Lâm Đồng           5
Đông Anh Hà Nội            4
An Dương  Hải Phòng        3
Hạ Long  Quảng Ninh        3
Bến Cát Bình Dương         2
Phan Thiết  Bình Thuận     2
Quy Nhơn  Bình Định        2
Đức Hòa  Long An           2
Dương Kinh  Hải Phòng      1
Rạch Giá  Kiên Giang       1
Đan Phượng Hà Nội          1
Từ Sơn  Bắc Ninh           1
Bà Rịa Bà Rịa Vũng Tàu     1
Tân Uyên Bình Dương        1
Thủy Nguyên  Hải Phòng     1
Bình Thủy  Cần Thơ         1
Hội An  Quảng Nam          1
Nhơn Trạch  Đồng Nai       1
Phước Ninh Đà Nẵng         1
La Gi  Bình Thuận          1
Bến Lức  Long An           1
Chương Mỹ Hà Nội           1
Phú Quốc  Kiên Giang       1
Mỹ Tho  Tiền Giang         1
Huế  Thừa Thiên Huế        1
Name: location, dtype: int64

In [ ]:
#Convert those <= 10 locations to 'other'
df2.location = df2.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df2.location.unique())
df2.head(5)


,total_site_area,property_name,number_of_bedrooms,city,district,price_unit,location
361,55,3,1,TP.Hồ Chí Minh,Quận 1,12,Quận 1 TP.Hồ Chí Minh
374,55,3,1,TP.Hồ Chí Minh,Quận 9,6,Quận 9 TP.Hồ Chí Minh
376,88,3,2,Hà Nội,Nam Từ Liêm,10,Nam Từ Liêm Hà Nội
382,72,3,2,TP.Hồ Chí Minh,Quận 9,7,Quận 9 TP.Hồ Chí Minh
385,78,3,3,TP.Hồ Chí Minh,Thủ Đức,6,Thủ Đức TP.Hồ Chí Minh


In [ ]:
bedroom_stats = df2.groupby('number_of_bedrooms')['number_of_bedrooms'].agg('count').sort_values(ascending=False)
bedroom_stats

number_of_bedrooms
2.0     5499
3.0     2174
1.0     1349
4.0      689
5.0      310
6.0      229
7.0       59
8.0       51
10.0      19
9.0       16
15.0      10
11.0       5
12.0       5
14.0       4
18.0       4
40.0       3
30.0       3
20.0       3
21.0       2
13.0       2
27.0       2
86.0       2
17.0       2
23.0       1
25.0       1
22.0       1
16.0       1
29.0       1
34.0       1
73.0       1
41.0       1
50.0       1
52.0       1
53.0       1
28.0       1
Name: number_of_bedrooms, dtype: int64

In [ ]:
#Remove outliers on number_of_bedrooms
df3 = df2[~(df2.number_of_bedrooms > 8)]
df3.shape

(10360, 7)

In [ ]:
df3.price_unit.max()

1800.0

In [ ]:
#Remove outliers based on distribution
def remove_punit_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_unit)
        st = np.std(subdf.price_unit)
        reduced_df = subdf[(subdf.price_unit > (m-st)) & (subdf.price_unit <= (m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index = True)
    return df_out

df4 = remove_punit_outliers(df3)
len(df4)

9461

**One hot encoding**


In [ ]:
dummies = pd.get_dummies(df4.location)
dummies.head(3)

,Ba Đình Hà Nội,Biên Hòa Đồng Nai,Bình Chánh TP.Hồ Chí Minh,Bình Thạnh TP.Hồ Chí Minh,Bình Tân TP.Hồ Chí Minh,Bắc Từ Liêm Hà Nội,Cầu Giấy Hà Nội,Dĩ An Bình Dương,Gia Lâm Hà Nội,Gò Vấp TP.Hồ Chí Minh,Hai Bà Trưng Hà Nội,Hoài Đức Hà Nội,Hoàn Kiếm Hà Nội,Hoàng Mai Hà Nội,Hà Đông Hà Nội,Hải Châu Đà Nẵng,Hồng Bàng Hải Phòng,Long Biên Hà Nội,Lê Chân Hải Phòng,Nam Từ Liêm Hà Nội,Ngô Quyền Hải Phòng,Nha Trang Khánh Hòa,Nhà Bè TP.Hồ Chí Minh,Phú Nhuận TP.Hồ Chí Minh,Quận 1 TP.Hồ Chí Minh,Quận 10 TP.Hồ Chí Minh,Quận 11 TP.Hồ Chí Minh,Quận 12 TP.Hồ Chí Minh,Quận 2 TP.Hồ Chí Minh,Quận 3 TP.Hồ Chí Minh,Quận 4 TP.Hồ Chí Minh,Quận 5 TP.Hồ Chí Minh,Quận 6 TP.Hồ Chí Minh,Quận 7 TP.Hồ Chí Minh,Quận 8 TP.Hồ Chí Minh,Quận 9 TP.Hồ Chí Minh,Sơn Trà Đà Nẵng,Thanh Khê Đà Nẵng,Thanh Trì Hà Nội,Thanh Xuân Hà Nội,Thuận An Bình Dương,Thủ Dầu Một Bình Dương,Thủ Đức TP.Hồ Chí Minh,Tân Bình TP.Hồ Chí Minh,Tân Phú TP.Hồ Chí Minh,Tây Hồ Hà Nội,Văn Giang Hưng Yên,Vũng Tàu Bà Rịa Vũng Tàu,other,Đống Đa Hà Nội
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df5 = pd.concat([df4, dummies.drop('other',axis='columns')],axis='columns')
df5.head(3)

,total_site_area,property_name,number_of_bedrooms,city,district,price_unit,location,Ba Đình Hà Nội,Biên Hòa Đồng Nai,Bình Chánh TP.Hồ Chí Minh,Bình Thạnh TP.Hồ Chí Minh,Bình Tân TP.Hồ Chí Minh,Bắc Từ Liêm Hà Nội,Cầu Giấy Hà Nội,Dĩ An Bình Dương,Gia Lâm Hà Nội,Gò Vấp TP.Hồ Chí Minh,Hai Bà Trưng Hà Nội,Hoài Đức Hà Nội,Hoàn Kiếm Hà Nội,Hoàng Mai Hà Nội,Hà Đông Hà Nội,Hải Châu Đà Nẵng,Hồng Bàng Hải Phòng,Long Biên Hà Nội,Lê Chân Hải Phòng,Nam Từ Liêm Hà Nội,Ngô Quyền Hải Phòng,Nha Trang Khánh Hòa,Nhà Bè TP.Hồ Chí Minh,Phú Nhuận TP.Hồ Chí Minh,Quận 1 TP.Hồ Chí Minh,Quận 10 TP.Hồ Chí Minh,Quận 11 TP.Hồ Chí Minh,Quận 12 TP.Hồ Chí Minh,Quận 2 TP.Hồ Chí Minh,Quận 3 TP.Hồ Chí Minh,Quận 4 TP.Hồ Chí Minh,Quận 5 TP.Hồ Chí Minh,Quận 6 TP.Hồ Chí Minh,Quận 7 TP.Hồ Chí Minh,Quận 8 TP.Hồ Chí Minh,Quận 9 TP.Hồ Chí Minh,Sơn Trà Đà Nẵng,Thanh Khê Đà Nẵng,Thanh Trì Hà Nội,Thanh Xuân Hà Nội,Thuận An Bình Dương,Thủ Dầu Một Bình Dương,Thủ Đức TP.Hồ Chí Minh,Tân Bình TP.Hồ Chí Minh,Tân Phú TP.Hồ Chí Minh,Tây Hồ Hà Nội,Văn Giang Hưng Yên,Vũng Tàu Bà Rịa Vũng Tàu,Đống Đa Hà Nội
0,78.42,3,2,Hà Nội,Ba Đình,22,Ba Đình Hà Nội,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,57,3,1,Hà Nội,Ba Đình,17,Ba Đình Hà Nội,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,120,3,3,Hà Nội,Ba Đình,30,Ba Đình Hà Nội,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df6 = df5.drop(['city','district','location'],axis='columns')
df6.head(3)

,total_site_area,property_name,number_of_bedrooms,price_unit,Ba Đình Hà Nội,Biên Hòa Đồng Nai,Bình Chánh TP.Hồ Chí Minh,Bình Thạnh TP.Hồ Chí Minh,Bình Tân TP.Hồ Chí Minh,Bắc Từ Liêm Hà Nội,Cầu Giấy Hà Nội,Dĩ An Bình Dương,Gia Lâm Hà Nội,Gò Vấp TP.Hồ Chí Minh,Hai Bà Trưng Hà Nội,Hoài Đức Hà Nội,Hoàn Kiếm Hà Nội,Hoàng Mai Hà Nội,Hà Đông Hà Nội,Hải Châu Đà Nẵng,Hồng Bàng Hải Phòng,Long Biên Hà Nội,Lê Chân Hải Phòng,Nam Từ Liêm Hà Nội,Ngô Quyền Hải Phòng,Nha Trang Khánh Hòa,Nhà Bè TP.Hồ Chí Minh,Phú Nhuận TP.Hồ Chí Minh,Quận 1 TP.Hồ Chí Minh,Quận 10 TP.Hồ Chí Minh,Quận 11 TP.Hồ Chí Minh,Quận 12 TP.Hồ Chí Minh,Quận 2 TP.Hồ Chí Minh,Quận 3 TP.Hồ Chí Minh,Quận 4 TP.Hồ Chí Minh,Quận 5 TP.Hồ Chí Minh,Quận 6 TP.Hồ Chí Minh,Quận 7 TP.Hồ Chí Minh,Quận 8 TP.Hồ Chí Minh,Quận 9 TP.Hồ Chí Minh,Sơn Trà Đà Nẵng,Thanh Khê Đà Nẵng,Thanh Trì Hà Nội,Thanh Xuân Hà Nội,Thuận An Bình Dương,Thủ Dầu Một Bình Dương,Thủ Đức TP.Hồ Chí Minh,Tân Bình TP.Hồ Chí Minh,Tân Phú TP.Hồ Chí Minh,Tây Hồ Hà Nội,Văn Giang Hưng Yên,Vũng Tàu Bà Rịa Vũng Tàu,Đống Đa Hà Nội
0,78.42,3,2,22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,57,3,1,17,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,120,3,3,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df6.shape

(9461, 53)

In [ ]:
X = df6.drop('price_unit', axis = 'columns')
X.head()
X.columns

Index(['total_site_area', 'property_name', 'number_of_bedrooms',
       'Ba Đình Hà Nội', 'Biên Hòa  Đồng Nai', 'Bình Chánh TP.Hồ Chí Minh',
       'Bình Thạnh TP.Hồ Chí Minh', 'Bình Tân TP.Hồ Chí Minh',
       'Bắc Từ Liêm Hà Nội', 'Cầu Giấy Hà Nội', 'Dĩ An Bình Dương',
       'Gia Lâm Hà Nội', 'Gò Vấp TP.Hồ Chí Minh', 'Hai Bà Trưng Hà Nội',
       'Hoài Đức Hà Nội', 'Hoàn Kiếm Hà Nội', 'Hoàng Mai Hà Nội',
       'Hà Đông Hà Nội', 'Hải Châu Đà Nẵng', 'Hồng Bàng  Hải Phòng',
       'Long Biên Hà Nội', 'Lê Chân  Hải Phòng', 'Nam Từ Liêm Hà Nội',
       'Ngô Quyền  Hải Phòng', 'Nha Trang  Khánh Hòa', 'Nhà Bè TP.Hồ Chí Minh',
       'Phú Nhuận TP.Hồ Chí Minh', 'Quận 1 TP.Hồ Chí Minh',
       'Quận 10 TP.Hồ Chí Minh', 'Quận 11 TP.Hồ Chí Minh',
       'Quận 12 TP.Hồ Chí Minh', 'Quận 2 TP.Hồ Chí Minh',
       'Quận 3 TP.Hồ Chí Minh', 'Quận 4 TP.Hồ Chí Minh',
       'Quận 5 TP.Hồ Chí Minh', 'Quận 6 TP.Hồ Chí Minh',
       'Quận 7 TP.Hồ Chí Minh', 'Quận 8 TP.Hồ Chí Minh',
       'Quận 9 TP.Hồ 

In [ ]:
y = df6.price_unit
y.head()

0    22
1    17
2    30
3    20
4    18
Name: price_unit, dtype: object

In [ ]:
#20% test - 80% train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)
lr_clf.score(X_test,y_test)

0.4690939532036724

In [ ]:
#K-fold cross val evaluation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
cross_val_score(LinearRegression(), X, y, cv = cv)

array([0.4996318 , 0.53155298, 0.58736995, 0.56398274, 0.49490644])

In [ ]:
#Lasso & Decision Tree
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params':{
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(scores, columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

,model,best_score,best_params
0,linear_regression,0.535489,{'normalize': False}
1,lasso,0.340211,"{'alpha': 1, 'selection': 'cyclic'}"
2,decision_tree,0.550932,"{'criterion': 'mse', 'splitter': 'best'}"


In [ ]:
def predict_price(location,total_site_area,property_name,number_of_bedrooms):
    loc_index = np.where(X.columns==location)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = total_site_area
    x[1] = property_name
    x[2] = number_of_bedrooms
    if loc_index >= 0:
        x[loc_index] =  1
    return lr_clf.predict([x])[0]

In [ ]:
predict_price('Ba Đình Hà Nội', 78.42, 3, 2)

14.57550360419527